# 📊 ETL con Pandas

Proceso ETL completo: Extraer, Transformar y Cargar datos en SQLite.


In [ ]:
# Imports
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import os
import glob
import warnings
warnings.filterwarnings('ignore')

# Cargar dataset limpio (buscar la versión más reciente)
# Buscar archivos con patrón gym_lifters_clean_*.csv o gym_lifters_clean.csv
data_path = None

# Buscar archivos numerados (versiones nuevas)
pattern1 = "../data/gym_lifters_clean_*.csv"
pattern2 = "data/gym_lifters_clean_*.csv"
files1 = glob.glob(pattern1)
files2 = glob.glob(pattern2)
all_files = files1 + files2

if all_files:
    # Ordenar por fecha de modificación y tomar el más reciente
    all_files.sort(key=os.path.getmtime, reverse=True)
    data_path = all_files[0]
    print(f"📂 Archivo encontrado: {data_path}")
else:
    # Buscar archivo sin numeración (versión antigua)
    if os.path.exists("../data/gym_lifters_clean.csv"):
        data_path = "../data/gym_lifters_clean.csv"
    elif os.path.exists("data/gym_lifters_clean.csv"):
        data_path = "data/gym_lifters_clean.csv"
    else:
        raise FileNotFoundError("No se encuentra gym_lifters_clean.csv. Ejecuta primero la celda de limpieza (Cell 12)")

df_clean = pd.read_csv(data_path)
print(f"✅ Dataset cargado: {len(df_clean):,} filas, {len(df_clean.columns)} columnas")
print(f"📁 Archivo: {os.path.basename(data_path)}")
display(df_clean.head())


PASO 1: EXTRACCIÓN (E)


NameError: name 'pd' is not defined

: 

## Transformación - Modelo Dimensional

Crear tablas de dimensiones y tabla de hechos (esquema estrella).


In [ ]:
# Crear dim_athlete
print("🔨 Creando dim_athlete...")
dim_athlete = df_clean[['athlete_id', 'name', 'gender', 'age', 'country']].copy()
dim_athlete = dim_athlete[dim_athlete['name'].notna() & (dim_athlete['name'] != '')]
dim_athlete = dim_athlete.drop_duplicates(subset=['name', 'country'], keep='first')
dim_athlete = dim_athlete.reset_index(drop=True)
dim_athlete.insert(0, 'id_athlete', range(1, len(dim_athlete) + 1))
dim_athlete['athlete_id'] = dim_athlete.apply(
    lambda row: row['athlete_id'] if pd.notna(row['athlete_id']) and row['athlete_id'] != '' 
    else f"ath_{row['id_athlete']}", axis=1
)
dim_athlete = dim_athlete[['id_athlete', 'athlete_id', 'name', 'gender', 'age', 'country']]
print(f"✅ dim_athlete: {len(dim_athlete):,} atletas únicos")
display(dim_athlete.head())


In [ ]:
# Crear dim_competition
print("\n🔨 Creando dim_competition...")
dim_competition = df_clean[['competition', 'year', 'category']].copy()
dim_competition = dim_competition[dim_competition['competition'].notna() & dim_competition['year'].notna()]
dim_competition = dim_competition.drop_duplicates(subset=['competition', 'year', 'category'], keep='first')
dim_competition = dim_competition.reset_index(drop=True)
dim_competition.insert(0, 'id_competition', range(1, len(dim_competition) + 1))
dim_competition = dim_competition[['id_competition', 'competition', 'year', 'category']]
print(f"✅ dim_competition: {len(dim_competition):,} competencias únicas")
display(dim_competition.head())


In [ ]:
# Crear dim_team
print("\n🔨 Creando dim_team...")
dim_team = df_clean[['team', 'coach']].copy()
dim_team = dim_team[dim_team['team'].notna() & (dim_team['team'] != '')]
dim_team = dim_team.drop_duplicates(subset=['team', 'coach'], keep='first')
dim_team = dim_team.reset_index(drop=True)
dim_team.insert(0, 'id_team', range(1, len(dim_team) + 1))
dim_team = dim_team[['id_team', 'team', 'coach']]
print(f"✅ dim_team: {len(dim_team):,} equipos únicos")
display(dim_team.head())


In [ ]:
# Crear fact_lifting
print("\n🔨 Creando fact_lifting...")
fact_base = df_clean.copy()

# Relacionar con dimensiones
fact_with_athlete = fact_base.merge(dim_athlete[['id_athlete', 'name', 'country']], on=['name', 'country'], how='left')
fact_with_competition = fact_with_athlete.merge(dim_competition[['id_competition', 'competition', 'year', 'category']], on=['competition', 'year', 'category'], how='left')
fact_with_team = fact_with_competition.merge(dim_team[['id_team', 'team', 'coach']], on=['team', 'coach'], how='left')

# Filtrar registros válidos
fact_lifting = fact_with_team[
    fact_with_team['id_athlete'].notna() &
    fact_with_team['id_competition'].notna() &
    fact_with_team['id_team'].notna()
].copy()

# Seleccionar columnas finales
fact_lifting = fact_lifting[['id_athlete', 'id_competition', 'id_team', 'snatch_kg', 'clean_and_jerk_kg', 'total_kg', 'body_weight_kg', 'event_rank', 'medal', 'record_status', 'lifting_style']]

print(f"✅ fact_lifting: {len(fact_lifting):,} registros ({len(fact_lifting)/len(fact_base)*100:.1f}% del total)")
display(fact_lifting.head())


## Carga - Guardar en SQLite

Cargar todas las tablas en `warehouse_pandas.db`


In [ ]:
# Preparar y cargar en SQLite
os.makedirs("warehouse", exist_ok=True)
db_path = "warehouse/warehouse_pandas.db"

if os.path.exists(db_path):
    os.remove(db_path)

engine = create_engine(f'sqlite:///{db_path}', echo=False)

# Cargar tablas
tables = [
    ("dim_athlete", dim_athlete),
    ("dim_competition", dim_competition),
    ("dim_team", dim_team),
    ("fact_lifting", fact_lifting)
]

print("📊 Cargando tablas en SQLite...")
for name, df in tables:
    df.to_sql(name=name, con=engine, if_exists="replace", index=False)
    print(f"   ✅ {name}: {len(df):,} registros")

print(f"\n✅ ETL completado! Base de datos: {db_path} ({os.path.getsize(db_path) / 1024:.2f} KB)")


In [ ]:
# Verificación final
conn = sqlite3.connect(db_path)
tables = ["dim_athlete", "dim_competition", "dim_team", "fact_lifting"]

print("\n📊 Resumen de tablas:")
for table in tables:
    count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn)['count'].iloc[0]
    print(f"   ✅ {table}: {count:,} registros")

print("\n📝 Muestras:")
for table in tables:
    display(pd.read_sql_query(f"SELECT * FROM {table} LIMIT 3", conn))

conn.close()
print("\n✅ ¡ETL completado!")


In [ ]:
# Fase 1: Limpieza de datos
import pandas as pd
import numpy as np
import os

# Cargar dataset
if os.path.exists("../data/gym_lifters.csv"):
    data_path = "../data/gym_lifters.csv"
elif os.path.exists("data/gym_lifters.csv"):
    data_path = "data/gym_lifters.csv"
else:
    raise FileNotFoundError("No se encuentra gym_lifters.csv")

df = pd.read_csv(data_path)
print(f"Dataset cargado: {df.shape[0]:,} filas, {df.shape[1]} columnas")
display(df.head())

# Exploración
print("\nValores nulos:")
print(df.isna().sum())
print(f"\nDuplicados: {df.duplicated().sum()}")

# Limpieza
df = df.drop_duplicates()
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.strip().str.lower()

valores_invalidos = ['?', 'n/a', 'na', 'unknown', '-1', 'none']
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].replace(valores_invalidos, np.nan)

# Limpiar columnas numéricas
columnas_peso = ['snatch_kg', 'clean_and_jerk_kg', 'total_kg', 'body_weight_kg']
for col in columnas_peso:
    if col in df.columns:
        df[col] = df[col].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

df["age"] = pd.to_numeric(df["age"], errors='coerce')
df["year"] = pd.to_numeric(df["year"], errors='coerce')
df["year"] = df["year"].where((df["year"] >= 1900) & (df["year"] <= 2025), np.nan)
df["event_rank"] = pd.to_numeric(df["event_rank"], errors='coerce')

# Rellenar valores faltantes
df["age"] = df["age"].fillna(df["age"].median())
for col in columnas_peso:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())
df["year"] = df["year"].fillna(df["year"].mode()[0] if not df["year"].mode().empty else np.nan)

# Guardar dataset limpio con numeración incremental
import glob
import re

# Determinar la ruta correcta de la carpeta data
if os.path.exists("../data"):
    data_dir = "../data"
elif os.path.exists("data"):
    data_dir = "data"
else:
    # Crear la carpeta data en la ubicación actual
    data_dir = "data"
    os.makedirs(data_dir, exist_ok=True)

# Asegurar que la carpeta data existe
os.makedirs(data_dir, exist_ok=True)

# Buscar archivos existentes con el patrón gym_lifters_clean_*.csv
pattern1 = f"{data_dir}/gym_lifters_clean_*.csv"
pattern2 = "data/gym_lifters_clean_*.csv" if data_dir != "data" else None
pattern3 = "../data/gym_lifters_clean_*.csv" if data_dir != "../data" else None

existing_files = glob.glob(pattern1)
if pattern2:
    existing_files.extend(glob.glob(pattern2))
if pattern3:
    existing_files.extend(glob.glob(pattern3))

# Eliminar duplicados
existing_files = list(set(existing_files))

# Encontrar el siguiente número disponible
if existing_files:
    # Extraer números de los archivos existentes
    numbers = []
    for file in existing_files:
        match = re.search(r'gym_lifters_clean_(\d+)\.csv', file)
        if match:
            numbers.append(int(match.group(1)))
    
    if numbers:
        next_number = max(numbers) + 1
    else:
        next_number = 1
else:
    next_number = 1

# Crear el nombre del archivo con el número
clean_path = f"{data_dir}/gym_lifters_clean_{next_number}.csv"
df.to_csv(clean_path, index=False)
print(f"\n✅ Dataset limpio guardado en: {clean_path}")
print(f"📊 Versión: {next_number}")
print(f"📁 Ruta absoluta: {os.path.abspath(clean_path)}")
display(df.sample(5))


FileNotFoundError: [Errno 2] No such file or directory: 'gym_lifters.csv'